In [1]:
import pandas as pd
import numpy as np
import collections

In [2]:
# File path for first csv
schools_data= "schools_complete.csv"
# Read our first csv into Pandas as a dataframe 
schools_data_df = pd.read_csv(schools_data)
# Rename "name" column to schools
schools_data_df = schools_data_df.rename(index=str, columns={"name": "school"})
# File path for second csv
student_data = "students_complete.csv"
# Read our second csv into Pandas 
student_data_df = pd.read_csv(student_data)

In [3]:
# Get a list of columns for reference, in first csv
schools_data_df.columns

Index(['School ID', 'school', 'type', 'size', 'budget'], dtype='object')

In [4]:
# Get a list of columns for reference, in 2nd csv
student_data_df.columns

Index(['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score',
       'math_score'],
      dtype='object')

In [5]:
# Merge Dataframes and view first 5 rows of new df 
merge_table_df = pd.merge(schools_data_df, student_data_df, how="left", on="school")
merge_table_df.head()

,School ID,school,type,size,budget,Student ID,name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [6]:
# Drop unnecessary columns from df
merge_table_df = merge_table_df.drop("Student ID", axis=1)
merge_table_df = merge_table_df.drop("School ID", axis=1)
merge_table_df.head()

,school,type,size,budget,name,gender,grade,reading_score,math_score
0,Huang High School,District,2917,1910635,Paul Bradley,M,9th,66,79
1,Huang High School,District,2917,1910635,Victor Smith,M,12th,94,61
2,Huang High School,District,2917,1910635,Kevin Rodriguez,M,12th,90,60
3,Huang High School,District,2917,1910635,Dr. Richard Scott,M,12th,67,58
4,Huang High School,District,2917,1910635,Bonnie Ray,F,9th,97,84


In [7]:
# Find total number of records
total_records = len(merge_table_df)

# Find total number of schools
number_of_schools = (merge_table_df["school"]).nunique()

# Find total count of number of students 
number_students = (merge_table_df["size"]).value_counts()
number_students = number_students.sum()

# Find the total budget 
reduced_df = merge_table_df.groupby("school").agg(np.mean)
total_budget = reduced_df["budget"].sum()

# Find the average math score
average_math_score = merge_table_df.math_score.mean()

# Find the average reading score
average_reading_score = merge_table_df.reading_score.mean()

# Find the percent passing math
percent_passing_math = merge_table_df[merge_table_df["math_score"]>= 70]
percent_passing_math = len(percent_passing_math)
percent_passing_math = (percent_passing_math/total_records) * 100

# Find the percent passing reading
percent_passing_reading = merge_table_df[merge_table_df["reading_score"]>= 70]
percent_passing_reading = len(percent_passing_reading)
percent_passing_reading = (percent_passing_reading/total_records) * 100

# Find the overall passing rate
overall_passing_rate = (percent_passing_math + percent_passing_reading)/2

In [8]:
# Create new dataframe from calculations
from collections import OrderedDict
district_summary_dict = {"Total Schools": [number_of_schools],
                        "Total Students": [number_students],
                        "Total Budget": [total_budget],
                        "Average Math Score": [average_math_score],
                        "Average Reading Score": [average_reading_score],
                        "Percent Passing Math":[percent_passing_math],
                        "Percent Passing Reading":[percent_passing_reading],
                        "Overall Passing Rate":[overall_passing_rate]}
district_summary_dict = OrderedDict(sorted(district_summary_dict.items()))
new_district_summary = OrderedDict([("Total Schools", [number_of_schools]),
                        ("Total Students", [number_students]),
                        ("Total Budget", [total_budget]),
                        ("Average Math Score", [average_math_score]),
                        ("Average Reading Score", [average_reading_score]),
                        ("Percent Passing Math", [percent_passing_math]),
                        ("Percent Passing Reading", [percent_passing_reading]),
                        ("Overall Passing Rate", [overall_passing_rate])])

district_summary_df = pd.DataFrame(new_district_summary)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
0,15,39170,24649428.0,78.985371,81.87784,74.980853,85.805463,80.393158


In [9]:
# Begin the schools summary by finding the average reading and math scores per school
new_merge_df = merge_table_df.groupby("school").mean()
new_merge_df
# Reset index
new_merge_df = new_merge_df.reset_index(level=None, drop=False, inplace=False)

# Add column with school type via merge
school_type_df = merge_table_df.loc[:,"school":"type"]
school_type_df = pd.DataFrame(school_type_df.groupby("school").min())
# Reset index so that we can use "school" as a merge column
school_type_df = school_type_df.reset_index(level=None, drop=False, inplace=False)
school_summary_table_df = pd.merge(school_type_df, new_merge_df, how="left", on="school")

# Create a new column called "Per Student Budget" and add to existing dataframe 
school_summary_table_df["Per Student Budget"] = school_summary_table_df["budget"]/school_summary_table_df["size"]

# Calculate and add a new column called percent passing reading
new_merge_df = merge_table_df[merge_table_df["reading_score"] >= 70]
group_of_passing_reading = new_merge_df.groupby("school").count()["name"]
school_percent_passing_reading = group_of_passing_reading / merge_table_df.groupby("school").count()["name"]*100
school_percent_passing_reading = school_percent_passing_reading.to_frame()
school_percent_passing_reading = school_percent_passing_reading.reset_index(level=None, drop=False, inplace=False)
school_summary_merge = pd.merge(school_summary_table_df, school_percent_passing_reading, how="left", on="school")

# Create a new column called "Percent Passing Math" and add to existing dataframe 
new_merge_df = merge_table_df[merge_table_df["math_score"] >= 70]
group_of_passing_math = new_merge_df.groupby("school").count()["name"]
school_percent_passing_math = group_of_passing_math / merge_table_df.groupby("school").count()["name"]*100
school_percent_passing_math = school_percent_passing_math.to_frame()
school_percent_passing_math = school_percent_passing_math.reset_index(level=None, drop=False, inplace=False)
school_summary_merge = pd.merge(school_summary_merge, school_percent_passing_math, how="left", on="school")

# Create a new column called "Overall Passing Rate" and add to existing dataframe 
school_summary_merge["Overall Passing Rate"] = (school_summary_merge['name_y']+school_summary_merge['name_x'])/2
# Rename columns
school_summary_merge = school_summary_merge.rename(columns={"name_x":"Percent Passing Reading","name_y":"Percent Passing Math"})
school_summary_merge

,school,type,size,budget,reading_score,math_score,Per Student Budget,Percent Passing Reading,Percent Passing Math,Overall Passing Rate
0,Bailey High School,District,4976.0,3124928.0,81.033963,77.048432,628.0,81.933280,66.680064,74.306672
1,Cabrera High School,Charter,1858.0,1081356.0,83.975780,83.061895,582.0,97.039828,94.133477,95.586652
2,Figueroa High School,District,2949.0,1884411.0,81.158020,76.711767,639.0,80.739234,65.988471,73.363852
3,Ford High School,District,2739.0,1763916.0,80.746258,77.102592,644.0,79.299014,68.309602,73.804308
4,Griffin High School,Charter,1468.0,917500.0,83.816757,83.351499,625.0,97.138965,93.392371,95.265668
5,Hernandez High School,District,4635.0,3022020.0,80.934412,77.289752,652.0,80.862999,66.752967,73.807983
6,Holden High School,Charter,427.0,248087.0,83.814988,83.803279,581.0,96.252927,92.505855,94.379391
7,Huang High School,District,2917.0,1910635.0,81.182722,76.629414,655.0,81.316421,65.683922,73.500171
8,Johnson High School,District,4761.0,3094650.0,80.966394,77.072464,650.0,81.222432,66.057551,73.639992
9,Pena High School,Charter,962.0,585858.0,84.044699,83.839917,609.0,95.945946,94.594595,95.270270


In [10]:
# Create a table that highlights the top 5 performing schools based on overall passing rate
top_schools = school_summary_merge.sort_values(by=['Overall Passing Rate'], ascending=False)
top_schools.head()

,school,type,size,budget,reading_score,math_score,Per Student Budget,Percent Passing Reading,Percent Passing Math,Overall Passing Rate
1,Cabrera High School,Charter,1858.0,1081356.0,83.975780,83.061895,582.0,97.039828,94.133477,95.586652
12,Thomas High School,Charter,1635.0,1043130.0,83.848930,83.418349,638.0,97.308869,93.272171,95.290520
9,Pena High School,Charter,962.0,585858.0,84.044699,83.839917,609.0,95.945946,94.594595,95.270270
4,Griffin High School,Charter,1468.0,917500.0,83.816757,83.351499,625.0,97.138965,93.392371,95.265668
13,Wilson High School,Charter,2283.0,1319574.0,83.989488,83.274201,578.0,96.539641,93.867718,95.203679


In [11]:
# Bottom performing schools
bottom_schools = school_summary_merge.sort_values(by=['Overall Passing Rate'], ascending=True)
bottom_schools.head()

,school,type,size,budget,reading_score,math_score,Per Student Budget,Percent Passing Reading,Percent Passing Math,Overall Passing Rate
10,Rodriguez High School,District,3999.0,2547363.0,80.744686,76.842711,637.0,80.220055,66.366592,73.293323
2,Figueroa High School,District,2949.0,1884411.0,81.158020,76.711767,639.0,80.739234,65.988471,73.363852
7,Huang High School,District,2917.0,1910635.0,81.182722,76.629414,655.0,81.316421,65.683922,73.500171
8,Johnson High School,District,4761.0,3094650.0,80.966394,77.072464,650.0,81.222432,66.057551,73.639992
3,Ford High School,District,2739.0,1763916.0,80.746258,77.102592,644.0,79.299014,68.309602,73.804308


In [12]:
# Math scores for each grade, grouped by school
student_data_df_9th = student_data_df[student_data_df["grade"] == '9th']
mathbygrade_9th = student_data_df_9th.groupby("school").math_score.mean()
mathbygrade_9th = mathbygrade_9th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_10th = student_data_df[student_data_df["grade"] == '10th']
mathbygrade_10th = student_data_df_10th.groupby("school").math_score.mean()
mathbygrade_10th = mathbygrade_10th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_11th = student_data_df[(student_data_df["grade"] == '11th')]
mathbygrade_df_11th = student_data_df_11th.groupby("school").math_score.mean()
mathbygrade_df_11th = mathbygrade_df_11th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_12th = student_data_df[(student_data_df["grade"] == '12th')]
mathbygrade_df_12th = student_data_df_12th.groupby("school").math_score.mean()
mathbygrade_df_12th = mathbygrade_df_12th.reset_index(level=None, drop=False, name=None, inplace=False)

# Merge all new data into one dataframe
merge_mathbygrade_df = pd.merge(mathbygrade_df_11th, mathbygrade_df_12th, how="outer", on= "school", suffixes=('_11th', '_12th'))
merge_mathbygrade_df = pd.merge(mathbygrade_10th, merge_mathbygrade_df, how= "outer", on= "school")
merge_mathbygrade_df = pd.merge(mathbygrade_9th, merge_mathbygrade_df, how= "outer", on= "school")
merge_mathbygrade_df
# Change column labels
merge_mathbygrade_df = merge_mathbygrade_df.rename(columns={"math_score_x":"math_score_9th","math_score_y":"math_score_10th"})
merge_mathbygrade_df

,school,math_score_9th,math_score_10th,math_score_11th,math_score_12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [13]:
# Reading scores by grade, grouped by school
student_data_df_9th = student_data_df[student_data_df["grade"] == '9th']
readingbygrade_9th = student_data_df_9th.groupby("school").reading_score.mean()
readingbygrade_9th = readingbygrade_9th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_10th = student_data_df[student_data_df["grade"] == '10th']
readingbygrade_10th = student_data_df_10th.groupby("school").reading_score.mean()
readingbygrade_10th = readingbygrade_10th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_11th = student_data_df[(student_data_df["grade"] == '11th')]
readingbygrade_11th = student_data_df_11th.groupby("school").reading_score.mean()
readingbygrade_11th = readingbygrade_11th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_12th = student_data_df[(student_data_df["grade"] == '12th')]
readingbygrade_12th = student_data_df_12th.groupby("school").reading_score.mean()
readingbygrade_12th = readingbygrade_12th.reset_index(level=None, drop=False, name=None, inplace=False)

# Merge all new data into one dataframe
merge_readingbygrade_df = pd.merge(readingbygrade_11th, readingbygrade_12th, how="outer", on= "school", suffixes=('_11th', '_12th'))
merge_readingbygrade_df = pd.merge(readingbygrade_10th, merge_readingbygrade_df, how= "outer", on= "school")
merge_readingbygrade_df = pd.merge(readingbygrade_9th, merge_readingbygrade_df, how= "outer", on= "school")
merge_readingbygrade_df
# Change column labels
merge_readingbygrade_df = merge_readingbygrade_df.rename(columns={"reading_score_x":"reading_score_9th","reading_score_y":"reading_score_10th"})
merge_readingbygrade_df

,school,reading_score_9th,reading_score_10th,reading_score_11th,reading_score_12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [14]:
# Scores by school spending per student budget
bins = [575, 600, 620, 640, 660]
spending_names = ["$575-599", "$600-619", "$620-639", "$640-659"]
school_summary_merge["Spending per student"] = pd.cut(school_summary_merge["Per Student Budget"], bins, labels=spending_names)
school_summary_reduced = school_summary_merge.drop("size", axis = 1)
school_summary_reduced = school_summary_reduced.drop("budget", axis = 1)
school_summary_reduced = school_summary_reduced.drop("Per Student Budget", axis = 1)
school_spending_group = school_summary_reduced.groupby("Spending per student")
school_spending_group.mean()

,reading_score,math_score,Percent Passing Reading,Percent Passing Math,Overall Passing Rate
Spending per student,,,,,
$575-599,83.892196,83.436210,96.459627,93.541501,95.000564
$600-619,84.044699,83.839917,95.945946,94.594595,95.270270
$620-639,82.120471,79.474551,87.468080,77.139934,82.304007
$640-659,80.957446,77.023555,80.675217,66.701010,73.688113


In [15]:
# Scores by school size
bins = [0, 1750, 3250, 5000]
group_names = ["Small", "Medium", "Large"]
school_summary_reduced["School Size Category"] = pd.cut(school_summary_merge["size"], bins, labels=group_names)
school_size_categories = school_summary_reduced.groupby("School Size Category")
school_size_categories.mean()

,reading_score,math_score,Percent Passing Reading,Percent Passing Math,Overall Passing Rate
School Size Category,,,,,
Small,83.881343,83.603261,96.661677,93.441248,95.051462
Medium,82.676142,80.545935,89.628554,82.169092,85.898823
Large,80.919864,77.063340,81.059691,66.464293,73.761992


In [16]:
# Scores by School type
school_type_categories = school_summary_reduced.groupby("type")
school_type_categories.mean() 

,reading_score,math_score,Percent Passing Reading,Percent Passing Math,Overall Passing Rate
type,,,,,
Charter,83.896421,83.473852,96.586489,93.620830,95.103660
District,80.966636,76.956733,80.799062,66.548453,73.673757
